In [ ]:
# ---------------------------------------------------------
# LIBRERÍAS
# ---------------------------------------------------------

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import difflib

# ---------------------------------------------------------
# 1. CARGA Y LIMPIEZA DE DATOS
# ---------------------------------------------------------

# Se carga el dataset de F1 limpio. 
df = pd.read_csv(
    'Data\\f1_clean_dataset.csv',
    na_values=['\\N', 'NULL', '', 'nan']
)

# Renombramos columnas importantes
df.rename(
    columns={
        'points': 'puntos', 
        'year': 'año', 
        'team_name': 'constructor', 
        'driver_fullname': 'nombre_piloto'}, 
        inplace=True)

# Convertimos puntos y año a números evitando errores
df[['puntos', 'año']] = (
    df[['puntos', 'año']]
    .apply(pd.to_numeric, errors='coerce')
    .fillna(0)
)

# Estándar: eliminar espacios, normalizar nombres
df['nombre_piloto'] = df['nombre_piloto'].astype(str).str.strip()
df['constructor']   = df['constructor'].astype(str).str.strip()


# ---------------------------------------------------------
# 2. CONSTRUCCIÓN DE LA TABLA DE TEMPORADA HISTÓRICA
# ---------------------------------------------------------

# Puntos totales por piloto/año
puntos_piloto = (
    df.groupby(['año', 'nombre_piloto'])['puntos']
    .sum()
    .reset_index()
)

# Ranking de pilotos por año
puntos_piloto['posicion'] = (
    puntos_piloto.groupby('año')['puntos']
    .rank(ascending=False, method='min')
)

# Equipo principal del piloto (Moda del equipo por piloto)
equipo_piloto = (
    df.groupby(['año', 'nombre_piloto'])['constructor']
    .agg(lambda x: x.mode().iat[0] if not x.mode().empty else None)
    .reset_index()
)

# Unimos puntos y equipos
temporada = pd.merge(puntos_piloto, equipo_piloto,
                     on=['año', 'nombre_piloto'])


# Ranking de constructores
puntos_const = (
    df.groupby(['año', 'constructor'])['puntos']
    .sum()
    .reset_index()
)

puntos_const['rank_team'] = (
    puntos_const.groupby('año')['puntos']
    .rank(ascending=False, method='min')
)

# Añadir ranking del constructor
temporada = pd.merge(
    temporada,
    puntos_const[['año', 'constructor', 'rank_team']],
    on=['año', 'constructor'],
    how='left'
)


# ---------------------------------------------------------
# 3. CREACIÓN DE DATOS ENTRENAMIENTO
# ---------------------------------------------------------

# Copiamos temporada
t_actual = temporada.copy()
t_prev   = temporada.copy()

# Año previo = actual - 1
t_prev['año'] += 1

# Renombramos columnas como anteriores
t_prev.rename(columns={
    'puntos': 'pts_ant',
    'posicion': 'pos_ant',
    'rank_team': 'rank_team_ant'
}, inplace=True)

# Merge entre temporada actual y datos del año previo
datos = pd.merge(
    t_actual,
    t_prev[['año', 'nombre_piloto', 'pts_ant', 'pos_ant', 'rank_team_ant']],
    on=['año', 'nombre_piloto'],
    how='inner'
)

# Eliminamos filas donde los valores previos sean NaN
datos = datos.dropna(subset=['pts_ant', 'pos_ant', 'rank_team_ant'])


# ---------------------------------------------------------
# 4. ENTRENAR MODELO RANDOM FOREST
# ---------------------------------------------------------

# Año más reciente en el dataset
ultimo_anio = datos['año'].max()

train = datos[datos['año'] < ultimo_anio]
test  = datos[datos['año'] == ultimo_anio]

X_train = train[['pts_ant', 'pos_ant', 'rank_team_ant']]
y_train = train['posicion']

X_test = test[['pts_ant', 'pos_ant', 'rank_team_ant']]
y_test = test['posicion']

# Entrenar modelo
rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    random_state=90
)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

# Evaluación
print("Evaluación para el año", ultimo_anio)
print("Error Medio Absoluto:", round(mean_absolute_error(y_test, y_pred), 2))
print("Error Cuadrático Medio:", round(np.sqrt(mean_squared_error(y_test, y_pred)), 2))
print("R Cuadrado:", round(r2_score(y_test, y_pred), 3))

Evaluación para el año 2024
MAE: 3.27
RMSE: 4.22
R2: 0.575


In [ ]:
# ---------------------------------------------------------
# 4.1 ENTRENAR MODELO CON TODAS LAS OBSERVACIONES
# ---------------------------------------------------------

# Variables predictoras
X = datos[['pts_ant', 'pos_ant', 'rank_team_ant']]

# Variable objetivo (posición del año actual)
y = datos['posicion']

# Modelo
rf = RandomForestRegressor(
    n_estimators=300,        
    max_depth=None,      
    random_state=90
)

rf.fit(X, y)

# ---------------------------------------------------------
# 5. CREAR PREDICCIONES PARA 2025 USANDO 2024
# ---------------------------------------------------------

datos_2024 = temporada[temporada['año'] == 2024].copy()

datos_2024.rename(columns={
    'puntos': 'pts_ant',
    'posicion': 'pos_ant',
    'rank_team': 'rank_team_ant'
}, inplace=True)

# Predecimos posición 2025
datos_2024['prediccion_2025'] = rf.predict(
    datos_2024[['pts_ant', 'pos_ant', 'rank_team_ant']]
)

In [9]:
# ---------------------------------------------------------
# 6. FUNCIÓN DE BÚSQUEDA DE PILOTO CON MATCH DIFUSO
# ---------------------------------------------------------

def obtener_prediccion(nombre_usuario: str):
    nombre_usuario = nombre_usuario.strip()

    # Lista de pilotos válidos
    lista_pilotos = datos_2024['nombre_piloto'].unique().tolist()

    # INTENTO 1: coincidencia exacta
    if nombre_usuario in lista_pilotos:
        match = nombre_usuario
        mensaje = f"Piloto encontrado: {match}"

    else:
        # INTENTO 2: coincidencia difusa
        coincidencias = difflib.get_close_matches(
            nombre_usuario, lista_pilotos, n=1, cutoff=0.4
        )

        if coincidencias:
            match = coincidencias[0]
            mensaje = f"Buscando a '{nombre_usuario}', mostrando resultado para: '{match}'"
        else:
            return f"Error: No se encontró ningún piloto similar a «{nombre_usuario}»."

    # Extraemos fila del piloto encontrado
    fila = datos_2024[datos_2024['nombre_piloto'] == match].iloc[0]

    # Formato amigable
    return (
        f"\n--- {mensaje} ---\n"
        f"Equipo: {fila['constructor']}\n"
        f"Posición 2024: {int(fila['pos_ant'])} (Puntos: {int(fila['pts_ant'])})\n"
        f"Predicción posición 2025: {fila['prediccion_2025']:.1f}\n"
    )

In [4]:
# ---------------------------------------------------------
# 7. INTERFAZ DE USUARIO
# ---------------------------------------------------------

print("""
      *#################*
         --F1Analyzer--
      *#################*

### Predicción de posición ###""")

entrada = input("Selecciona un piloto: ")

print(obtener_prediccion(entrada))



      *#################*
         --F1Analyzer--
      *#################*

### Predicción de posición ###

--- Buscando a 'Verstapen', mostrando resultado para: 'Max Verstappen' ---
Equipo: Red Bull
Posición 2024: 1 (Puntos: 399)
Predicción posición 2025: 3.4



## Top 10 Predicho para la Temporada 2025

In [10]:
# Reiniciar indices
datos_2024.reset_index(drop=True, inplace=True)

# Organizar pilotos por posición predicha
datos_2024 = datos_2024.sort_values('prediccion_2025')

print("\nPosiciones Predichas para 2025:\n")
print(datos_2024[['nombre_piloto', 'prediccion_2025']].head(10))


Posiciones Predichas para 2025:

           nombre_piloto  prediccion_2025
16    Michael Schumacher         1.290000
23    Rubens Barrichello         3.151921
1        David Coulthard         3.690000
13    Juan Pablo Montoya         3.793778
21       Ralf Schumacher         3.994444
11         Jenson Button         4.616921
17         Mika Häkkinen         5.111444
9           Jarno Trulli         8.267611
18         Nick Heidfeld         9.043889
6   Giancarlo Fisichella        10.361226
